# Example: Combining Technical Repeats into Single Data object

You can now store multiple technical repeats into a single data object.

> **Note**:
> You can combine any two hdf5 files into technical repeats so long as their base acquisition parameters are the same.
> In this example we will use 4 different files, which are not actually technical repeats.
> This is for demonstration purposes, in general, you should only combine files that are actually acquired with the same sample and measuremtn parameters.

So first let's import FRETBursts

In [ ]:
from fretbursts import *
sns = init_notebook(apionly=True)

Now we download the files:

In [ ]:
urls = ['https://zenodo.org/record/5902313/files/HP3_TE150_SPC630.hdf5',
        'https://zenodo.org/record/5902313/files/HP3_TE200_SPC630.hdf5',
        'https://zenodo.org/record/5902313/files/HP3_TE250_SPC630.hdf5',
        'https://zenodo.org/record/5902313/files/HP3_TE300_SPC630.hdf5']
for url in urls:
    download_file(url, save_dir='./data')

## Loading muliple photon-HDF5 files

To load mutliple files into a single data object (as technical repeats), simple provide the filenames as a list to `loader.photon_hdf5()`.

In [ ]:
filenames = ["data/HP3_TE150_SPC630.hdf5", 
             "data/HP3_TE200_SPC630.hdf5", 
             "data/HP3_TE250_SPC630.hdf5", 
             "data/HP3_TE300_SPC630.hdf5"]
d = loader.photon_hdf5(filenames)

Now the rest of the analysis can proceed, what you do to one technical repeat, you do to the other.

In [ ]:
loader.alex_apply_period(d)
d.calc_bg(bg.exp_fit, time_s=1000, tail_min_us=(800, 4000, 1500, 1000, 3000))
d.burst_search(L=10, m=10, F=6)
ds = d.select_bursts(select_bursts.size, add_naa=True, th1=30)

The technical repeats are stored like separate spots in a multi-spot experiment, so now if you want to look at the E-values of a given repeat, it looks like `d.E[i]` where `i` is the index of the repeat in the data, as you supplied it to `loader.photon_hdf5`

In [ ]:
ds.E

When you plot with `dplot()` the default will be to display each separately, in a 8-channel plot

In [ ]:
dplot(ds, hist2d_alex)

If you want to plot the combined results, just add the keyword argument `i=None`

In [ ]:
dplot(ds, hist2d_alex, i=None)

The fitter objects also now fit not just individual repeats, but the entire concatenated set:

In [ ]:
E_fitter = bext.bursts_fitter(ds, 'E', binwidth=0.03)
model = mfit.factory_two_gaussians()
E_fitter.fit_histogram(model=model, pdf=False, method='leastsq')
dplot(ds, hist_fret, show_model=True, pdf=False, i=0);

Finally, in the fitter object, new attributes have been added, basically, if there is a per-channel parameter, then the same name with `_tot` added to the end will contain the summed counts, pdf, or fitting object.

In [ ]:
E_fitter.fit_res_tot